# Práctica 3 - Diffie Hellman

- Diego García Díaz
- Alberto Pérez Álvarez

## Generación de claves con Diffie Hellman

In [146]:
import random, math

def es_primo(numero):
  """
  Verifica si un número entero es primo.
  True si el número es primo, False de lo contrario.
  """
  if not isinstance(numero, int):
      raise TypeError("El input debe ser un número entero.")
  if numero <= 1:
    return False
  if numero <= 3:
      return True # 2 y 3 son primos
  if numero % 2 == 0 or numero % 3 == 0:
      return False # Eliminar múltiplos de 2 y 3
  # Solo necesitamos comprobar divisores de la forma 6k ± 1 hasta sqrt(n)
  i = 5
  while i * i <= numero:
    if numero % i == 0 or numero % (i + 2) == 0:
      return False
    i += 6
  return True

def genera_un_primo_aleatorio(max_val, min_val=1, intentos_max=10000):
    """
    Función auxiliar para generar un único primo aleatorio en el rango [min_val, max_val).
    """
    if min_val >= max_val:
        raise ValueError("El valor mínimo debe ser estrictamente menor que el máximo.")

    for _ in range(intentos_max):
        candidato = random.randrange(min_val, max_val)
        if es_primo(candidato):
            return candidato

    # Si llegamos aquí, no se encontró un primo en los intentos dados
    raise ValueError(f"No se pudo encontrar un primo en el rango [{min_val}, {max_val}) "
                     f"después de {intentos_max} intentos.")


El codigo de arriba permite crear un numero primo aleatorio dado un limite superior, y se puede modificar el inferior y los intentos, en caso de que el numero a generar sea muy grande.

In [147]:
def exponencia_modular(base, expo, mod):
    if not isinstance(base, int) or not isinstance(expo, int) or not isinstance(mod, int):
        raise TypeError("Las entradas deben ser int")
    if expo < 0:
        raise ValueError("El exponente no puede ser negativo para este algoritmo")
    if mod <= 0:
        raise ValueError("El módulo debe ser un positivo")

    i = expo  
    x = base % mod 
    r = 1  

    while i > 0:
        if i % 2 != 0:
            r = (r * x) % mod 
            
        x = (x * x) % mod
        i = i // 2 

    return r

Implementacion de la version iterativa mejorada de las diapositivas (diapo 37)

In [202]:
def diffie_hellman():
    p_primo = genera_un_primo_aleatorio(1000000000)

    n_primo = genera_un_primo_aleatorio(p_primo)
    print(f"El valor de p = {p_primo} \nEl valor de la base = {n_primo}")

    random.seed(55)

    clave_privada_Alice = random.randrange(1,p_primo)
    

    clave_publica_Alice = exponencia_modular(n_primo, clave_privada_Alice, p_primo)
    print(f"\nEl mensaje de Alice a Bob es {clave_publica_Alice}")

    clave_privada_Bob = random.randrange(1,p_primo)

    clave_publica_Bob = exponencia_modular(n_primo, clave_privada_Bob, p_primo)
    print(f"El mensaje de Bob a Alice es {clave_publica_Bob}")

    print("\nLa clave privada de Alice es",clave_privada_Alice)
    print("La clave privada de Bob es",clave_privada_Bob)

    clave_mensaje_Alice = exponencia_modular(clave_publica_Bob, clave_privada_Alice, p_primo)

    clave_mensaje_Bob = exponencia_modular(clave_publica_Alice, clave_privada_Bob, p_primo)

    print(f"\nLas clave del mensaje en ambos lados es igual == {clave_mensaje_Bob == clave_mensaje_Alice}")
    print(f"La clave == {clave_mensaje_Alice} = {clave_mensaje_Bob}")

    return (clave_publica_Alice,clave_publica_Bob)
    



Implementación del procedimiento Diffie-Hellman. Ejemplo abajo:

In [203]:
diffie_hellman()

El valor de p = 711491147 
El valor de la base = 506765719

El mensaje de Alice a Bob es 118334279
El mensaje de Bob a Alice es 213545450

La clave privada de Alice es 97001677
La clave privada de Bob es 210718381

Las clave del mensaje en ambos lados es igual == True
La clave == 492447201 = 492447201


(118334279, 213545450)

In [ ]:
def generar_numero_de_n_digitos(n):
  """Genera un numero de N digitos y lo devuelve
  """
  random.seed(55)
  if not isinstance(n, int) or n <= 0:
    print("Error: n tiene que ser positivo.")
    return None

  if n == 1:
    return str(random.randint(0, 9))  # Pequeño

  # Generar primer digito
  primer_digito = str(random.randint(1, 9)) # No puede empezar por 0

  # Generar el resto de digitos
  tolresto_digitos = ''.join(str(random.randint(0, 9)) for _ in range(n - 1))

  return int(primer_digito + tolresto_digitos)

In [ ]:
def diffie_hellman_DHGroup_18():
    '''
    Algoritmo de Diffie Hellman pero usando el grupo 18 para una conexión segura
    Definido en https://datatracker.ietf.org/doc/html/rfc3526#section-7 por el IETF
    
    p = 2^8192 - 2^8128 - 1 + 2^64 * { [2^8062 pi] + 4743158 }
    g = 2
    Las claves privadas deben tener una longitud de 8192*log10(2) = 2466 digitos
    '''
    # 2^8192 - 2^8128 - 1 + 2^64 * { [2^8062 pi] + 4743158 }
    p = 4260734904763343474415214567811658204484980956848328952242582449086524097354631402455204975972018819412938141285454816286033979502635459620078102600724364193734988460823915382971476955108832160341063796187431387810451953905706920362236486440611541026827811510847592810122953074775473890151248349684175046214374685261484212840185706897329762372919919242797759881543785829207036631215813076421026877183051106746774039120996620676269984436626245665218596476607867803598759873201050064528591145849621542098407657860475472730296228037560470579997081050262970165567795299225762198463891475443671171369665318601202395527944156057794028252548456621030740203806568954311732417505336759799403373704595758324340243503271441405374000182553327450153888099766985995597088436403494899700977292516366494314611903230419110002707752869745866823971275688008831922630001238951939136607632633443149186558959841894340299631161570599536403500160942513734977613454645537998829256578591641732345516821109882909819843676312140730405735179567649093045628324213475368816046690284785009394288186678511395341235762607246352280530654957560666209982186794845006769241676051313070645806180994643536939054967199445514196130416830872291194934373470796189506756348190074719613675655216603593564079094524464116527724118209571796942491863184859901744432348856785224143868569064934438816208872289811275149411350337552266527523814261537476911623204238426923714567949664173068544736660008102771242780568116808500050358395918744837580483289198829341269113462874067985679447992734522843870766767747162830209531377490777737400399888837809775287884330831611019322183698852805452742997604257682648197012071519110598518880335545183865879569985508518348726810008097876862483307652319596792352081624928802898869497890053730393205770216565027643520610283979368258045719211723681190240281960838994269299412919439862430051258623556060988056569037208411097803033625563526564039329160951026895687092143055400154641292662431613209789820102520638326905985417632070782127846892165509726958647618927935441267301112383277679138532690431054976285478645967700843895050720077657002098327272896719602457783580669541927773743618311320691973016680091056472620692407741265010630436069785112102788657925640022376954668847225631841209904635085874626893125004097660890355212716528901396634945423116475654633619293205602958310596962441679333882638350769356026859679445374334434787163400202054177220448504981127301099825113836355059711
    print("El modulo es",p)

    g = 2

    clave_privada_Alice = generar_numero_de_n_digitos(2466)

    clave_publica_Alice = exponencia_modular(g, clave_privada_Alice, p)
    print(f"\nEl mensaje de Alice a Bob es {clave_publica_Alice}")

    clave_privada_Bob = generar_numero_de_n_digitos(2466)

    clave_publica_Bob = exponencia_modular(g, clave_privada_Bob, p)
    print(f"El mensaje de Bob a Alice es {clave_publica_Bob}")

    print("\nLa clave privada de Alice es",clave_privada_Alice)
    print("La clave privada de Bob es",clave_privada_Bob)

    clave_mensaje_Alice = exponencia_modular(clave_publica_Bob, clave_privada_Alice, p)

    clave_mensaje_Bob = exponencia_modular(clave_publica_Alice, clave_privada_Bob, p)

    print(f"\nLas clave del mensaje en ambos lados es igual --> {clave_mensaje_Bob == clave_mensaje_Alice}")
    print(f"La clave de Alice ({clave_mensaje_Alice}) es igual a la de Bob ({clave_mensaje_Bob})")
    return (clave_publica_Alice,clave_publica_Bob)
    

In [176]:
diffie_hellman_DHGroup_18()

El modulo es 426073490476334347441521456781165820448498095684832895224258244908652409735463140245520497597201881941293814128545481628603397950263545962007810260072436419373498846082391538297147695510883216034106379618743138781045195390570692036223648644061154102682781151084759281012295307477547389015124834968417504621437468526148421284018570689732976237291991924279775988154378582920703663121581307642102687718305110674677403912099662067626998443662624566521859647660786780359875987320105006452859114584962154209840765786047547273029622803756047057999708105026297016556779529922576219846389147544367117136966531860120239552794415605779402825254845662103074020380656895431173241750533675979940337370459575832434024350327144140537400018255332745015388809976698599559708843640349489970097729251636649431461190323041911000270775286974586682397127568800883192263000123895193913660763263344314918655895984189434029963116157059953640350016094251373497761345464553799882925657859164173234551682110988290981

(289718589125924687708622196819054398937959489360970522904834620706795867302644973448024977704276726371582830749621485805315159396734874798665657044651305730545142340196040112814953225323819349169466514161783654826204982896169799661483663303291797000617805290441975969515473061908772305785950979619401699663701833270868960487299784841286747198582222200229620770575161454979295965968639553902327426884367345517960963151945761895899204416344890919515268464235214996083845016556242441860816219621929310803781353966473623161434689084154808151571875447494367671757149266923120801310315046198139689980784181290010845866073413231800707458847594682257213651773875139818378644057811196563013836076231675524486437275849262229304630195454229094944084711614109684218754147208444725561602865326263628840187655835447719060309113467613694980210498234892047116492403684730692461599936590923508998524727872065684205535891342834377374648031658045471627384603793242359345603159321369703637562380943227807238798159746427

## Fuerza Bruta

In [206]:
def mostrarTiempo(tiempo):  # Para imprimir los tiempos con un formato bonito
    horas = int(tiempo // 3600)
    minutos = int((tiempo % 3600) // 60)
    segundos = int(tiempo % 60)
    ms = int((tiempo - int(tiempo)) * 1000000)
    return f"{horas:01d}:{minutos:02d}:{segundos:02d}.{ms:06d}"

In [168]:
import time

def fuerza_bruta(g,p,mensaje_Alice,mensaje_Bob):
    t_inicio = time.time()
    exponente = 0
    resultado = -1
    soluciones = []
    while len(soluciones) < 2:
        exponente += 1
        resultado = exponencia_modular(g,exponente,p)
        if resultado == mensaje_Alice:
            soluciones.append(("Alice",exponente,time.time()-t_inicio))

        elif resultado == mensaje_Bob:
            soluciones.append(("Bob",exponente,time.time()-t_inicio))

    return soluciones

In [ ]:
'''
Ejemplo pequeño
'''
g = 4
p = 19
clave_Alice = 8
clave_Bob = 9
publicacion_Alice = exponencia_modular(g,clave_Alice,p)
publicacion_Bob = exponencia_modular(g,clave_Bob,p)
soluciones = fuerza_bruta(g,p,publicacion_Alice,publicacion_Bob)
print("¡Se han encontrado las claves privadas!")
print(f"Primera clave obtenida: \n   De {soluciones[0][0]}\n   Clave: {soluciones[0][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[0][2])}")
print(f"Segunda clave obtenida: \n   De {soluciones[1][0]}\n   Clave: {soluciones[1][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[1][2])}")




¡Se han encontrado las claves privadas!
Primera clave obtenida: 
   De Alice
   Clave: 8
   Tiempo empleado: 0:00:00.000007
Segunda clave obtenida: 
   De Bob
   Clave: 9
   Tiempo empleado: 0:00:00.000010


In [205]:
'''
PRIMER EJEMPLO DE LA PRÁCTICA
'''

g = 12345701
p = 666666653
publicacion_Alice = 518633585
publicacion_Bob = 654181592
print("\nBuscando claves privadas...")
soluciones = fuerza_bruta(g,p,publicacion_Alice,publicacion_Bob)
print("¡Se han encontrado las claves privadas!")
print(f"Primera clave obtenida: \n   De {soluciones[0][0]}\n   Clave: {soluciones[0][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[0][2])}")
print(f"Segunda clave obtenida: \n   De {soluciones[1][0]}\n   Clave: {soluciones[1][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[1][2])}")


Buscando claves privadas...
¡Se han encontrado las claves privadas!


NameError: name 'mostrarTiempo' is not defined

In [ ]:
'''
Valores del grupo 18
'''
g = 2
p = 4260734904763343474415214567811658204484980956848328952242582449086524097354631402455204975972018819412938141285454816286033979502635459620078102600724364193734988460823915382971476955108832160341063796187431387810451953905706920362236486440611541026827811510847592810122953074775473890151248349684175046214374685261484212840185706897329762372919919242797759881543785829207036631215813076421026877183051106746774039120996620676269984436626245665218596476607867803598759873201050064528591145849621542098407657860475472730296228037560470579997081050262970165567795299225762198463891475443671171369665318601202395527944156057794028252548456621030740203806568954311732417505336759799403373704595758324340243503271441405374000182553327450153888099766985995597088436403494899700977292516366494314611903230419110002707752869745866823971275688008831922630001238951939136607632633443149186558959841894340299631161570599536403500160942513734977613454645537998829256578591641732345516821109882909819843676312140730405735179567649093045628324213475368816046690284785009394288186678511395341235762607246352280530654957560666209982186794845006769241676051313070645806180994643536939054967199445514196130416830872291194934373470796189506756348190074719613675655216603593564079094524464116527724118209571796942491863184859901744432348856785224143868569064934438816208872289811275149411350337552266527523814261537476911623204238426923714567949664173068544736660008102771242780568116808500050358395918744837580483289198829341269113462874067985679447992734522843870766767747162830209531377490777737400399888837809775287884330831611019322183698852805452742997604257682648197012071519110598518880335545183865879569985508518348726810008097876862483307652319596792352081624928802898869497890053730393205770216565027643520610283979368258045719211723681190240281960838994269299412919439862430051258623556060988056569037208411097803033625563526564039329160951026895687092143055400154641292662431613209789820102520638326905985417632070782127846892165509726958647618927935441267301112383277679138532690431054976285478645967700843895050720077657002098327272896719602457783580669541927773743618311320691973016680091056472620692407741265010630436069785112102788657925640022376954668847225631841209904635085874626893125004097660890355212716528901396634945423116475654633619293205602958310596962441679333882638350769356026859679445374334434787163400202054177220448504981127301099825113836355059711
dh = diffie_hellman_DHGroup_18()
print("\nBuscando claves privadas...")
publicacion_Alice = dh[0]
publicacion_Bob = dh[1]
soluciones = fuerza_bruta(g,p,publicacion_Alice,publicacion_Bob)
print("¡Se han encontrado las claves privadas!")
print(f"Primera clave obtenida: \n   De {soluciones[0][0]}\n   Clave: {soluciones[0][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[0][2])}")
print(f"Segunda clave obtenida: \n   De {soluciones[1][0]}\n   Clave: {soluciones[1][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[1][2])}")

El modulo es 426073490476334347441521456781165820448498095684832895224258244908652409735463140245520497597201881941293814128545481628603397950263545962007810260072436419373498846082391538297147695510883216034106379618743138781045195390570692036223648644061154102682781151084759281012295307477547389015124834968417504621437468526148421284018570689732976237291991924279775988154378582920703663121581307642102687718305110674677403912099662067626998443662624566521859647660786780359875987320105006452859114584962154209840765786047547273029622803756047057999708105026297016556779529922576219846389147544367117136966531860120239552794415605779402825254845662103074020380656895431173241750533675979940337370459575832434024350327144140537400018255332745015388809976698599559708843640349489970097729251636649431461190323041911000270775286974586682397127568800883192263000123895193913660763263344314918655895984189434029963116157059953640350016094251373497761345464553799882925657859164173234551682110988290981

KeyboardInterrupt: 

In [ ]:
'''
Primera implementación
'''
g = 506765719
p = 711491147
dh = diffie_hellman()
print("\nBuscando claves privadas...")
publicacion_Alice = dh[0]
publicacion_Bob = dh[1]
soluciones = fuerza_bruta(g,p,publicacion_Alice,publicacion_Bob)
print("¡Se han encontrado las claves privadas!")
print(f"Primera clave obtenida: \n   De {soluciones[0][0]}\n   Clave: {soluciones[0][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[0][2])}")
print(f"Segunda clave obtenida: \n   De {soluciones[1][0]}\n   Clave: {soluciones[1][1]}\n   Tiempo empleado: {mostrarTiempo(soluciones[1][2])}")

El valor de p = 711491147 
El valor de la base = 506765719

El mensaje de Alice a Bob es 118334279
El mensaje de Bob a Alice es 213545450

La clave privada de Alice es 97001677
La clave privada de Bob es 210718381

Las clave del mensaje en ambos lados es igual == True
La clave == 492447201 = 492447201

Buscando claves privadas...
¡Se han encontrado las claves privadas!
Primera clave obtenida: 
   De Alice
   Clave: 97001677
   Tiempo empleado: 0:04:28.758059
Segunda clave obtenida: 
   De Bob
   Clave: 210718381
   Tiempo empleado: 0:10:15.949985
